# <b>MODIS Water Validation Notebook - Compare models</b>

Purpose: Used to perform validation of C61 MOD44W products from different models. Compares those products to the previous version, C6 MOD44W.

*Note: We are following an incremental development lifecycle. This notebook is the first rendition which fit most of the requirements. Expect incremental releases which continue towards the goal of fully meeting requirements and increasing capabilities of the user.*

Date: 03/04/2024

*For DSG internal use*

## Tile and year selection

Choose which tile (see MODIS grid) and which year. Reference the grid image. 

The `h` followed by two numerical digits represent the <b>horizontal</b> tile ID. Use the column space to determine this ID. 

The `v` followed by two numerical digits represent the <b>vertical</b> tile ID. Use the row space to determine this ID. 

For example, the tile that is 9 columns to the right and 5 rows down is `h09v05`.

Example:
```python
TILE = 'h09v05'
```

![MODIS Grid Overlay](../imgs/modis_overlay.png)

In [1]:
# !pip install localtileserver

In [2]:
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl
from localtileserver import TileClient, get_leaflet_tile_layer 
from ipyleaflet import LegendControl, FullScreenControl, Popup
import matplotlib.colors as mcolors
from ipysheet import from_dataframe
import matplotlib.pyplot as plt
import ipywidgets as widgets
import rioxarray as rxr
from osgeo import gdal
from glob import glob
import xarray as xr
import numpy as np
import tempfile
import ipysheet
import warnings
import joblib
import os

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

### Data Parameters

In [3]:
# TILE = 'h12v09'
TILE = 'h22v01'
# TILE = 'h21v10'
# TILE = 'h09v05'

In [4]:
YEAR = 2019

In [5]:
target = False
eb = False
percent = False
noc = True
plot_obs = False
qa = False

In [7]:
############
#Directory Paths
############
#RFA dir paths
data_dir = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Burke_MW_RFA'

TAR_PATH = f'{data_dir}/v4_sample' #targeted rfa trained using v4.2.1 data
EB_PATH = f'{data_dir}/v2_cluster' #even balance trained rfa
MATCH_PATH = f'{data_dir}/v2_match' #random subset match size of even balance rfa
NOC_PATH = f'{data_dir}/v2_total' #rfa trained using v2.0.1 data

#MOD44W dir paths
MOD44W_C6_BASEPATH = '/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs/'
CACHE_DIR = '.cache'
os.makedirs(CACHE_DIR, exist_ok=True)

############
#Specific File Paths
############
#RFA file paths
file_qa_path = f'*{YEAR}*{TILE}*ProductQA.*tif'
file_path = f'*{YEAR}*{TILE}*Product.*tif'
file_sumobs_path = f'*{YEAR}*{TILE}*SumWater*tif'
file_ogmask_path = f'*{YEAR}*{TILE}*-Mask*tif'
#C6 file paths
mod44w_c6_path = f'{MOD44W_C6_BASEPATH}/{str(YEAR)}/MOD44W_{TILE}_{YEAR}_v5.tif'
print(mod44w_c6_path)
if not os.path.exists(mod44w_c6_path):
    raise FileNotFoundError(f'Could not find the MOD44W C6 file: {mod44w_c6_path}')

############
#CMAPS and legend
############
water_c6_cmap: list = [mcolors.cnames['white'], mcolors.cnames['white']]
water_rfa_qa_cmap: list = ['#ee82ee', '#FCB900', '#FF6900', '#800080']
qa_water_legend_dict = {
    'QA- Perm Water Flipped L->W': '#ee82ee', 
    'QA- Ocean Mask L->W': '#FCB900',
    'QA- Burn Scar W->L': '#FF6900',
    'QA- DEM Slope Change W->L': '#800080'}


############
#Geotransform parameters
############
crs = 'PROJCS["Sinusoidal",GEOGCS["Sphere",DATUM["Sphere",SPHEROID["Sphere",6371000,0]],PRIMEM["Greenwich",0],' + \
    'UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"]' + \
    ',PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0]' + \
    ',UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
mod44w_cs_ds = gdal.Open(mod44w_c6_path)
transform = mod44w_cs_ds.GetGeoTransform()

############
#KWARGS
############
temporary_files = []
rfa_kwargs = {'nodata':0,'show':False,'vmin':0,'vmax':1,'max_zoom':20}
obs_kwargs = {'nodata':0,'show':False,'vmin':0,'vmax':365,'max_zoom':20}  
pw_kwargs = {'nodata':0,'show':False,'vmin':1,'vmax':4,'max_zoom':20}  
transform_kwargs = {'transform':transform, 'projection':crs, 'year':YEAR, 
                        'tile':TILE, 'files_to_rm':temporary_files}


/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs//2019/MOD44W_h22v01_2019_v5.tif


### Plots

#### Functions

In [8]:
def parse_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 0, 0, -1)
    qa_array_parsed = xr.where(qa_array == 4, 1, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 3, qa_array_parsed)
    return qa_array_parsed

def parse_fix_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 2, 1, 0)
    qa_array_parsed = xr.where(qa_array == 4, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 3, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 4, qa_array_parsed)
    return qa_array_parsed

def open_and_write_temp(data_array, transform, projection, 
    year, tile, name = None, files_to_rm = None) -> str:
    tmpdir = tempfile.gettempdir()
    name_to_use = data_array.name if not name else name
    tempfile_name = f'MOD44W.A{year}001.{tile}.061.{name_to_use}.tif'
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    print(glob(tempfile_fp))
    if os.path.exists(tempfile_fp): 
        os.remove(tempfile_fp)
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(tempfile_fp, 4800, 4800, 
                          1, gdal.GDT_Float32, 
                          options=['COMPRESS=LZW'])
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(projection)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(data_array.data[0, :, :])
    outBand.SetNoDataValue(250)
    outDs.FlushCache()
    outDs = None
    outBand = None
    driver = None
    return tempfile_fp

def get_location(cache_dir: str, tile: str, def_location: list) -> list:
    cache_fp = os.path.join(cache_dir, f'{tile}.marker.location.sv')
    if os.path.exists(cache_fp):
        location = joblib.load(cache_fp)
    else:
        location = def_location
    return location

def cache_location(tile: str, location: list) -> None:
    cache_fp = os.path.join(CACHE_DIR, f'{tile}.marker.location.sv')
    output = joblib.dump(location, cache_fp)
    return None

def initialize_marker(tile: str, location: list, cache_dir: str) -> Marker:
    name = 'Location Marker'
    title = name
    location = get_location(cache_dir, tile, location)
    marker = Marker(name=name, title=name, location=location)
    return marker

def initialize_message(location: list) -> widgets.HTML:
    ll_message = widgets.HTML()
    ll_message.value = str(location)
    return ll_message

In [9]:
def data_plot(product_path, legend_name, data_color = None, pw = False, obs = False):
    """
    In 
        product_path (str): path to the tif rf prediction files
        legend_name (str): name of the product that is plotted
        data_color (str): the cmap color that the data will be plotted in
        pw (bool): if True then create QA layer, if false create RF product layer
        obs (bool): if True then create obs layer
    
    Returns
        rfa_water_mask_layer (leaflet_tile_layer): Map layer of the RFA Product 
        rfa_legend_dict (dict): Dictionary of the RFA Product cmap 
        perm_water_layer (leaflet_tile_layer): Map layer of the Permanent Water QA Product 
        
    """
    print(product_path)
    
    data_array = rxr.open_rasterio(sorted(glob(product_path))[0])
    if obs is True: print('Max',data_array.max(),'\nMin',data_array.min())
    if pw is True:
        product_water_array = parse_fix_qa(data_array)
        product_kwargs = pw_kwargs
        tile_name = f'{legend_name} QA'
        tile_cmap = water_rfa_qa_cmap
        perm_mask_name = f'{legend_name} perm mask'
        qa_mask_name = perm_mask_name.replace(' ','_')
    else:
        tile_name = f'{legend_name}'
        product_kwargs = rfa_kwargs
        
        if obs is True: 
            tile_cmap = 'jet'
            product_kwargs = obs_kwargs
        else: 
            product_water_array = data_array
            tile_cmap = [mcolors.cnames[data_color], mcolors.cnames[data_color]]
            product_kwargs = rfa_kwargs

        mask_name =  f'{legend_name} mask'
        qa_mask_name = mask_name.replace(' ','_')

    product_qa = open_and_write_temp(product_water_array, name=qa_mask_name, **transform_kwargs)
    product_client = TileClient(product_qa)
    product_layer = get_leaflet_tile_layer(
            product_client, cmap=tile_cmap, 
            name=tile_name, **product_kwargs)
  
    if (pw is True) or (obs is True): return product_layer
    else: 
        product_legend_dict = {tile_name: mcolors.cnames[data_color]}
        return product_layer, product_legend_dict

#### Using Functions

In [10]:
#c6 data
c6_client = TileClient(mod44w_c6_path)
c6_water_mask_layer = get_leaflet_tile_layer( c6_client, cmap=water_c6_cmap, name=f'C6', **rfa_kwargs)
c6_legend_dict = {'C6': mcolors.cnames['white']}

In [11]:
if target:
    tar_layer, tar_dict = data_plot(f'{TAR_PATH}/{file_path}','v421','goldenrod')
    if qa: 
        tar_qa_layer = data_plot(f'{TAR_PATH}/{file_qa_path}','v421', pw=True)
if eb:
    ebc_layer, ebc_dict = data_plot(f'{EBC_PATH}/{file_path}','EB cluster','plum')
    ebm_layer, ebm_dict = data_plot(f'{EBM_PATH}/{file_path}','EB match','darkorchid')
    if plot_obs:
        ebc_sumobs_layer = data_plot(f'{EBC_PATH}/{file_sumobs_path}','EB SumObs', obs=True)
    if qa: 
        ebc_qa_layer = data_plot(f'{EBC_PATH}/{file_qa_path}','EB cluster', pw=True)
        ebm_qa_layer = data_plot(f'{EBM_PATH}/{file_qa_path}','EB match', pw=True)
if percent: 
    pm_layer, pm_dict = data_plot(f'{PM_PATH}/{file_path}','P match','darkblue')
    pc_layer, pc_dict = data_plot(f'{PC_PATH}/{file_path}','P cluster','lightblue')
    if plot_obs: 
        pc_sumobs_layer = data_plot(f'{PC_PATH}/{file_sumobs_path}','P SumObs', obs=True)
    if qa: 
        pc_qa_layer = data_plot(f'{PC_PATH}/{file_qa_path}','P cluster', pw=True)
        pm_qa_layer = data_plot(f'{PM_PATH}/{file_qa_path}','P match', pw=True)
if noc: 
    noc_layer, noc_dict = data_plot(f'{NOC_PATH}/{file_path}','v201','black')
    if qa: 
        noc_qa_layer = data_plot(f'{NOC_PATH}/{file_qa_path}','v201', pw=True)

/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Burke_MW_RFA/not_outlier_pts/v2_total/*2019*h22v01*Product.*tif


IndexError: list index out of range

In [ ]:
legend_dict = {}
legend_dict.update(c6_legend_dict)
if target: 
    legend_dict.update(tar_dict)
if eb:
    legend_dict.update(ebc_dict)
    legend_dict.update(ebm_dict)
if percent:
    legend_dict.update(pc_dict)
    legend_dict.update(pm_dict)
if noc: 
    legend_dict.update(noc_dict)
legend_dict.update(qa_water_legend_dict)
legend = LegendControl(legend_dict)

In [ ]:
#########
#Setup map
#########
m = Map(
    center=c6_client.center(),
    zoom=c6_client.default_zoom,
    basemap=basemaps.Esri.WorldImagery,
    scroll_wheel_zoom=True,
    keyboard=True,
    layout=widgets.Layout(height='600px')
)
marker_location = c6_client.center()
marker = initialize_marker(tile=TILE, location=marker_location, cache_dir=CACHE_DIR)
latlon_message = initialize_message(marker.location)

def handle_click(**kwargs):
    latlon_message.value = str(marker.location)
    marker.popup = latlon_message
    cache_location(tile=TILE, location=marker.location)

m.add_layer(marker)

if target:
    m.add_layer(tar_layer)
    if qa:
        m.add_layer(tar_qa_layer)
if percent:
    m.add_layer(pm_layer)
    m.add_layer(pc_layer)
    if plot_obs: 
        m.add_layer(pc_sumobs_layer)
    if qa:
        m.add_layer(pm_qa_layer)
        m.add_layer(pc_qa_layer)
if eb:
    m.add_layer(ebm_layer)
    m.add_layer(ebc_layer)
    if plot_obs: 
        m.add_layer(ebc_sumobs_layer)
    if qa:
        m.add_layer(ebm_qa_layer)
        m.add_layer(ebc_qa_layer)
if noc:
    m.add_layer(noc_layer)
    if qa:
        m.add_layer(noc_qa_layer)
    
m.add_layer(c6_water_mask_layer)
    
marker.on_click(handle_click)
# m.add_control(legend)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

In [ ]:
display(m)

## Save notes

Run this cell to save notes in the current working directory

In [ ]:
userid = !whoami
notes_path = f'../notes/{TILE}-{userid[0]}-notes.csv'
if os.path.exists(notes_path):
    notes_df = pd.read_csv(notes_path)
    notes_df = notes_df.drop(columns=['Unnamed: 0'])
    sheet_notes = ipysheet.from_dataframe(notes_df)
else:
    tile = [' ' for _ in range(75)]
    year = [' ' for _ in range(75)]
    location = [' ' for _ in range(75)]
    note = [' ' for _ in range(75)]
    data = {'Tile': tile, 'Year': year, 'Location': location, 'Note': note}
    notes_df = pd.DataFrame(data=data)
    sheet_notes = ipysheet.from_dataframe(notes_df)
sheet_notes.column_width = [3,3,4,10]
sheet_notes.layout = widgets.Layout(width='100%',height='100%')
sheet_notes

In [ ]:
sheet_notes_df = ipysheet.to_dataframe(sheet_notes)
sheet_notes_df.to_csv(notes_path)

### <b>DO NOT RUN THIS CELL UNTIL FINISHED WITH VALIDATION</b>
*Note: This will shut down the local tile servers*

*Ignore warnings as such:*
```
Server for key (default) not found.
```

In [ ]:
for path_to_delete in temporary_files_to_delete:
    if os.path.exists(path_to_delete):
        os.remove(path_to_delete)
    temporary_files_to_delete.remove(path_to_delete)

mod44w_rfa_v1_water_client.shutdown(True)